In [1]:
#import libraries for coding
import busio ##black library
import board ##black library
import adafruit_ccs811 ##black library
import time ##black and blue library
import smbus ##blue libarary
##!!!!!import gspread ##google sheets library

In [10]:
#google sheet code DON'T RUN RIGHT NOW
from oauth2client.service_account import ServiceAccountCredentials

# use creds to create a client to interact with the Google Drive API
scopes = ["https://www.googleapis.com/auth/drive", "https://www.googleapis.com/auth/drive.file", "https://www.googleapis.com/auth/spreadsheets"]
creds = ServiceAccountCredentials.from_json_keyfile_name('client_secret.json', scopes)
client = gspread.authorize(creds)

# Find a workbook by name and open the first sheet
# Make sure you use the right name here.
sheet = client.open("PiLog").sheet1

FileNotFoundError: [Errno 2] No such file or directory: 'client_secret.json'

In [3]:
#rowcode DON'T RUN RIGHT NOW
row = ["hi", "this", "is", "KT"]
index = 1
sheet.insert_row(row, index)

{'spreadsheetId': '1A_DiqnAGVyuwDIVRMzVv74I6P5NEaYuZ2CjlNNvSfFs',
 'updates': {'spreadsheetId': '1A_DiqnAGVyuwDIVRMzVv74I6P5NEaYuZ2CjlNNvSfFs',
  'updatedRange': 'Sheet1!A1:D1',
  'updatedRows': 1,
  'updatedColumns': 4,
  'updatedCells': 4}}

In [4]:
sheet.append_row(row) #DON'T RUN RIGHT NOW

{'spreadsheetId': '1A_DiqnAGVyuwDIVRMzVv74I6P5NEaYuZ2CjlNNvSfFs',
 'tableRange': 'Sheet1!A1:D1',
 'updates': {'spreadsheetId': '1A_DiqnAGVyuwDIVRMzVv74I6P5NEaYuZ2CjlNNvSfFs',
  'updatedRange': 'Sheet1!A2:D2',
  'updatedRows': 1,
  'updatedColumns': 4,
  'updatedCells': 4}}

In [5]:
# DON'T RUN RIGHT NOW
for i in range(10):
    row=[i,2*i,i-7]
    sheet.append_row(row)

In [2]:
#black sensor code
from board import *
i2c = board.I2C()   # uses board.SCL and board.SDA

In [3]:
#black sensor code
ccs =  adafruit_ccs811.CCS811(i2c)

In [4]:
#black sensor code - print readings
while True:
    print("CO2: ", ccs.eco2, " TVOC:", ccs.tvoc)
    print(ccs.data_ready)
    time.sleep(3)

CO2:  None  TVOC: None
False
CO2:  400  TVOC: 0
False


KeyboardInterrupt: 

In [5]:
#blue sensor code - read blue sensor data and calculate the ext. temp.
def readTemp():
    # By default the address of LM75A is set to 0x48
    address = 0x48

    # Read I2C data and calculate temperature
    bus = smbus.SMBus(1)
    raw = bus.read_word_data(address, 0) & 0xFFFF
    raw = ((raw << 8) & 0xFF00) + (raw >> 8)
    temperature = (raw / 32.0) / 8.0
    return temperature

In [6]:
#blue sensor code - contact and sign into Adafruit
from Adafruit_IO import Client, Feed, RequestError
from dotenv import load_dotenv
import os

#Load login token from .env file
load_dotenv()

ADAFRUIT_IO_USERNAME = os.environ.get("ADAFRUIT_IO_USERNAME")
ADAFRUIT_IO_KEY = os.environ.get("ADAFRUIT_IO_KEY")

print(ADAFRUIT_IO_USERNAME)
print(ADAFRUIT_IO_KEY)

aio = Client(ADAFRUIT_IO_USERNAME, ADAFRUIT_IO_KEY)

javalemon
aio_eZeA96r9ApV0ft5gO0oNUFKvoCAr


In [7]:
#blue sensor code - bme code
from bme280 import BME280
bme280 = BME280()

In [8]:
#blue sensor code - read feeds, create metadata, create letter variables, print data
exterFeed = aio.feeds("external")
tempFeed = aio.feeds("temperature")
humidFeed = aio.feeds("humidity")
pressFeed = aio.feeds("pressure")
co2Feed = aio.feeds("co2")
tvocFeed = aio.feeds("tvoc")

metadata = {"lat": 40.262726,
            "lon": -74.94198,
            "ele": 0,
            "created_at": None}
while True:
    e = readTemp()
    t = bme280.get_temperature()
    f = e * (9./5.) +32
    q = t * (9./5.) +32 
    h = bme280.get_humidity()
    p = bme280.get_pressure()
    co2 = ccs.eco2
    tvoc = ccs.tvoc
    aio.send_data(exterFeed.key, f, metadata)
    aio.send_data(tempFeed.key, q, metadata)
    aio.send_data(humidFeed.key, h, metadata)
    aio.send_data(pressFeed.key, p, metadata)
    aio.send_data(co2Feed.key, co2, metadata)
    aio.send_data(tvocFeed.key, tvoc, metadata)
    print(f, ":°Fahrenheit|")
    print(q, ":°Fahrenheit|")
    print(h, ":Humidity|")
    print(p, ":Barometric Pressure|")
    print(co2, ": PPM (CO2)|")
    print(tvoc, ":PPB (TVOC)|")
    print()
    time.sleep(15)

71.375 :°Fahrenheit|
69.16508733144147 :°Fahrenheit|
88.2681598298333 :Humidity|
684.1512265828492 :Barometric Pressure|
400 : PPM (CO2)|
0 :PPB (TVOC)|

71.375 :°Fahrenheit|
78.95720777748477 :°Fahrenheit|
48.222547415224426 :Humidity|
1007.3925919786357 :Barometric Pressure|
400 : PPM (CO2)|
0 :PPB (TVOC)|

72.275 :°Fahrenheit|
79.3291085990498 :°Fahrenheit|
54.658423378069514 :Humidity|
1007.428249246851 :Barometric Pressure|
456 : PPM (CO2)|
8 :PPB (TVOC)|



KeyboardInterrupt: 